<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_11_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# GAN(Generative Adversarial Networks)을 이용한 MNIST 데이터 생성 - Keras API를 이용한 구현
# Reference : https://github.com/TengdaHan/GAN-TensorFlow

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [ ]:
# MNIST 데이터를 다운로드 합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
# 28*28 형태의 이미지를 784차원으로 flattening 합니다.
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize합니다.
x_train, x_test = x_train / 255., x_test / 255.

# 생성된 MNIST 이미지를 8x8 Grid로 보여주는 plot 함수를 정의합니다.
def plot(samples):
  fig = plt.figure(figsize=(8, 8))
  gs = gridspec.GridSpec(8, 8)
  gs.update(wspace=0.05, hspace=0.05)

  for i, sample in enumerate(samples):
    ax = plt.subplot(gs[i])
    plt.axis('off')
    plt.imshow(sample.reshape(28, 28))

  return fig

# 학습에 필요한 설정값들을 선언합니다.
num_epoch = 100000
batch_size = 64
num_input = 28 * 28
num_latent_variable = 100   # 잠재 변수 z의 차원
num_hidden = 128
learning_rate = 0.001

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices(x_train)
train_data = train_data.repeat().shuffle(60000).batch(batch_size)
train_data_iter = iter(train_data)

def random_normal_intializer(stddev):
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=stddev, seed=None)

In [ ]:
# tf.keras.Model을 이용해서 Generator 모델을 정의합니다.
# Inputs:
#   X : 인풋 Latent Variable
# Outputs:
#   generated_mnist_image : 생성된 MNIST 이미지
class Generator(tf.keras.Model):
  def __init__(self):
    super(Generator, self).__init__()
    # 100 -> 128 -> 784
    # 히든 레이어 파라미터
    self.hidden_layer_1 = tf.keras.layers.Dense(num_hidden,
                                                activation='relu',
                                                kernel_initializer=random_normal_intializer(5e-2),
                                                bias_initializer=tf.keras.initializers.Constant(value=0.1))
    # 아웃풋 레이어 파라미터
    self.output_layer = tf.keras.layers.Dense(num_input,
                                              activation='sigmoid',
                                              kernel_initializer=random_normal_intializer(5e-2),
                                              bias_initializer=tf.keras.initializers.Constant(value=0.1))

  def call(self, x):
    hidden_layer = self.hidden_layer_1(x)
    generated_mnist_image = self.output_layer(hidden_layer)

    return generated_mnist_image

In [ ]:
# tf.keras.Model을 이용해서 Discriminator 모델을 정의합니다.
# Inputs:
#   X : 인풋 이미지
# Outputs:
#   predicted_value : Discriminator가 판단한 True(1) or Fake(0)
#   logits : sigmoid를 씌우기전의 출력값
class Discriminator(tf.keras.Model):
  def __init__(self):
    super(Discriminator, self).__init__()
    # 784 -> 128 -> 1
    # 히든 레이어 파라미터
    self.hidden_layer_1 = tf.keras.layers.Dense(num_hidden,
                                                activation='relu',
                                                kernel_initializer=random_normal_intializer(5e-2),
                                                bias_initializer=tf.keras.initializers.Constant(value=0.1))
    # 아웃풋 레이어 파라미터
    self.output_layer = tf.keras.layers.Dense(1,
                                              activation=None,
                                              kernel_initializer=random_normal_intializer(5e-2),
                                              bias_initializer=tf.keras.initializers.Constant(value=0.1))

  def call(self, x):
    hidden_layer = self.hidden_layer_1(x)
    logits = self.output_layer(hidden_layer)
    predicted_value = tf.nn.sigmoid(logits)

    return predicted_value, logits

In [ ]:
# Generator의 손실 함수를 정의합니다.
@tf.function
def generator_loss(D_fake_logits):
  return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake_logits)))         # log(D(G(z))

# Discriminator의 손실 함수를 정의합니다.
@tf.function
def discriminator_loss(D_real_logits, D_fake_logits):
  d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real_logits)))  # log(D(x))
  d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake_logits)))  # log(1-D(G(z)))
  d_loss = d_loss_real + d_loss_fake  # log(D(x)) + log(1-D(G(z)))

  return d_loss

In [ ]:
# 생성자(Generator) 모델을 선언합니다.
Generator_model = Generator()

# 구분자(Discriminator) 모델을 선언합니다.
Discriminator_model = Discriminator()

# Discriminator와 Generator를 위한 Optimizer들을 정의합니다.
discriminator_optimizer = tf.optimizers.Adam(learning_rate)
generator_optimizer = tf.optimizers.Adam(learning_rate)

# Discriminator 최적화를 위한 function을 정의합니다.
@tf.function
def d_train_step(discriminator_model, real_image, fake_image):
  with tf.GradientTape() as disc_tape:
    D_real, D_real_logits = discriminator_model(real_image)  # D(x)
    D_fake, D_fake_logits = discriminator_model(fake_image)  # D(G(z))
    loss = discriminator_loss(D_real_logits, D_fake_logits)
  gradients = disc_tape.gradient(loss, discriminator_model.trainable_variables)
  discriminator_optimizer.apply_gradients(zip(gradients, discriminator_model.trainable_variables))

# Generator 최적화를 위한 function을 정의합니다.
@tf.function
def g_train_step(generator_model, discriminator_model, z):
  with tf.GradientTape() as gen_tape:
    G = generator_model(z)
    D_fake, D_fake_logits = discriminator_model(G)  # D(G(z))
    loss = generator_loss(D_fake_logits)
  gradients = gen_tape.gradient(loss, generator_model.trainable_variables)
  generator_optimizer.apply_gradients(zip(gradients, generator_model.trainable_variables))

In [ ]:
# 생성된 이미지들을 저장할 generated_output 폴더를 생성합니다.
num_img = 0
if not os.path.exists('generated_output/'):
  os.makedirs('generated_output/')

# num_epoch 횟수만큼 최적화를 수행합니다.
for i in range(num_epoch):
  # MNIST 이미지를 batch_size만큼 불러옵니다.
  batch_X = next(train_data_iter)

  # Latent Variable의 인풋으로 사용할 noise를 Uniform Distribution에서 batch_size 개수만큼 샘플링합니다.
  batch_noise = np.random.uniform(-1., 1., [batch_size, 100]).astype('float32')

  # 500번 반복할때마다 생성된 이미지를 저장합니다.
  if i % 500 == 0:
    samples = Generator_model(np.random.uniform(-1., 1., [64, 100]).astype('float32')).numpy()
    fig = plot(samples)
    plt.savefig('generated_output/%s.png' % str(num_img).zfill(3), bbox_inches='tight')
    num_img += 1
    plt.close(fig)

  # Discriminator 최적화를 수행하고 Discriminator의 손실함수를 return합니다.
  _, d_loss_print = d_train_step(Discriminator_model, batch_X, Generator_model(batch_noise)), discriminator_loss(Discriminator_model(batch_X)[1], Discriminator_model(Generator_model(batch_noise))[1])

  # Generator 최적화를 수행하고 Generator 손실함수를 return합니다.
  _, g_loss_print = g_train_step(Generator_model, Discriminator_model, batch_noise), generator_loss(Discriminator_model(Generator_model(batch_noise))[1])

  # 100번 반복할때마다 Discriminator의 손실함수와 Generator 손실함수를 출력합니다.
  if i % 100 == 0:
    print('반복(Epoch): %d, Generator 손실함수(g_loss): %f, Discriminator 손실함수(d_loss): %f' % (i, g_loss_print, d_loss_print))

반복(Epoch): 0, Generator 손실함수(g_loss): 1.487437, Discriminator 손실함수(d_loss): 0.966135
반복(Epoch): 100, Generator 손실함수(g_loss): 2.496813, Discriminator 손실함수(d_loss): 0.412196
반복(Epoch): 200, Generator 손실함수(g_loss): 3.374493, Discriminator 손실함수(d_loss): 0.131666
반복(Epoch): 300, Generator 손실함수(g_loss): 4.530323, Discriminator 손실함수(d_loss): 0.013044
반복(Epoch): 400, Generator 손실함수(g_loss): 4.645432, Discriminator 손실함수(d_loss): 0.018580
반복(Epoch): 500, Generator 손실함수(g_loss): 5.229744, Discriminator 손실함수(d_loss): 0.047073
반복(Epoch): 600, Generator 손실함수(g_loss): 5.919028, Discriminator 손실함수(d_loss): 0.009279
반복(Epoch): 700, Generator 손실함수(g_loss): 5.749841, Discriminator 손실함수(d_loss): 0.028784
반복(Epoch): 800, Generator 손실함수(g_loss): 5.909723, Discriminator 손실함수(d_loss): 0.012418
반복(Epoch): 900, Generator 손실함수(g_loss): 5.823652, Discriminator 손실함수(d_loss): 0.004316
반복(Epoch): 1000, Generator 손실함수(g_loss): 7.779797, Discriminator 손실함수(d_loss): 0.003470
반복(Epoch): 1100, Generator 손실함수(g_loss): 9.4

KeyboardInterrupt: ignored